In [1]:
pip install -q -u bitsandbytes trl



Usage:   
  /usr/bin/python3 -m pip install [options] <requirement specifier> [package-index-options] ...
  /usr/bin/python3 -m pip install [options] -r <requirements file> [package-index-options] ...
  /usr/bin/python3 -m pip install [options] [-e] <vcs project url> ...
  /usr/bin/python3 -m pip install [options] [-e] <local project path> ...
  /usr/bin/python3 -m pip install [options] <archive url/path> ...

no such option: -u
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os 
os.listdir('/kaggle/working/')

In [ ]:
%%writefile train_dpo.py
from datasets import load_dataset

# Load dataset
ds = load_dataset("Jennny/helpsteer2-helpfulness-preference", split="train")

# Transform function
def transform(example):
    prompt = example["chosen"][0]["content"]
    chosen = example["chosen"][-1]["content"]
    rejected = example["rejected"][-1]["content"]

    return {
        "prompt": prompt,
        "chosen": chosen,
        "rejected": rejected,
        "margin": example.get("margin", 0)
    }

# Convert
dataset = ds.map(transform, remove_columns=ds.column_names)

from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import DPOConfig, DPOTrainer

model_name = "Qwen/Qwen3-0.6B"

# --- LOAD FROM CHECKPOINT ---
checkpoint_path = "/kaggle/working/Qwen3-0.6B-DPO/checkpoint-300"

model = AutoModelForCausalLM.from_pretrained(
    model_name
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

training_args = DPOConfig(
    output_dir="Qwen3-0.6B-DPO",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    save_steps=50,
    save_total_limit=1,
    report_to="none",
)

trainer = DPOTrainer(
    model=model,
    args=training_args,
    processing_class=tokenizer,
    train_dataset=dataset,
)

# ---- RESUME TRAINING ----
trainer.train(resume_from_checkpoint=checkpoint_path)


trainer.save_model("Qwen3-0.6B-DPO1")
tokenizer.save_pretrained("Qwen3-0.6B-DPO1")


In [ ]:
%%writefile multi_gpu.yaml
compute_environment: LOCAL_MACHINE
debug: false
deepspeed_config:
  gradient_accumulation_steps: 8
  gradient_clipping: 1.0
  offload_optimizer_device: none
  offload_param_device: none
  zero3_init_flag: false
  zero_stage: 2
distributed_type: DEEPSPEED
downcast_bf16: 'no'
dynamo_config:
  dynamo_backend: INDUCTOR
enable_cpu_affinity: false
machine_rank: 0
main_training_function: main
mixed_precision: 'no'
num_machines: 1
num_processes: 2
rdzv_backend: static
same_network: true
tpu_env: []
tpu_use_cluster: false
tpu_use_sudo: false
use_cpu: false

In [ ]:
!accelerate launch --config_file /kaggle/working/multi_gpu.yaml train_dpo.py

In [ ]:


from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("Qwen3-0.6B-DPO1")
tokenizer = AutoTokenizer.from_pretrained("Qwen3-0.6B-DPO1")


repo_id = "AIPlans/Qwen3-0.6B-DPO_NOTLORA"

model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)